# Перевод документа (doc-translator)

Запуск перевода так же, как через консоль: Pandoc, модель translategemma, целевой язык Russian.

- Чтобы перевести другой файл, измените `INPUT_DOCX` в ячейке с параметрами (например, `"1.docx"`).
- Чтобы задать свой системный промпт: установите `SYSTEM_PROMPT` в строку с текстом (можно взять за основу `DEFAULT_MARKDOWN_PROMPT` из следующей ячейки). Оставьте `None` для промпта по умолчанию.

In [ ]:
from pathlib import Path
import sys

sys.path.insert(0, str(Path(".").resolve() / "src"))

from doc_translator.processors.pandoc_utils import check_pandoc
from doc_translator.processors.pandoc_word import PandocWordProcessor
from doc_translator.translator.ollama import OllamaTranslator

In [ ]:
INPUT_DOCX = "3. PSR-L08-0623_Protocol_1.0 dated 10.06.2024_clean_EN_fin.docx"
LANGUAGE = "Russian"
MODEL = "translategemma:latest"
MAX_CHARS = 4000
TIMEOUT = 1800

# Системный промпт для модели: None = использовать встроенный (Markdown). Своя строка = заменить промпт.
SYSTEM_PROMPT = None

In [ ]:
# Промпт по умолчанию (Markdown). Можно скопировать в SYSTEM_PROMPT и отредактировать.
DEFAULT_MARKDOWN_PROMPT = """You are a professional translator. The input is a fragment of a document in Markdown format.
Translate ONLY the natural language text to the target language. You MUST:
- Keep all Markdown syntax exactly as in the input: headers (# ## ###), lists (- * 1.), bold/italic (** *), code blocks (```), links, etc.
- Tables: preserve EXACTLY. Both grid tables (lines starting with +, |, or =) and pipe tables (| col | col |). Do not add or remove any border characters, rows, or columns. Do NOT insert blank lines inside a table — tables must have no empty lines between rows. Only translate the text inside cells (after | and >).
- Do not add or remove any structural markup; only translate the visible text content.
- Output valid Markdown with the same structure. Do not add explanations or comments."""

In [ ]:
input_path = Path(INPUT_DOCX)
if not input_path.exists():
    raise FileNotFoundError(f"Файл не найден: {input_path}")

if not check_pandoc():
    raise RuntimeError("Pandoc не найден. Установите: https://pandoc.org/installing.html")

print(f"Документ: {input_path}")
print(f"Целевой язык: {LANGUAGE}")
print(f"Модель: {MODEL}")
print("Режим: Pandoc (Markdown ↔ docx со стилями)")
print(f"Максимум символов на чанк: {MAX_CHARS}")
print("-" * 50)

In [ ]:
translator_kw = dict(
    model=MODEL,
    base_url="http://localhost:11434",
    timeout=TIMEOUT,
    use_markdown_prompt=SYSTEM_PROMPT is None,
)
if SYSTEM_PROMPT is not None:
    translator_kw["system_prompt"] = SYSTEM_PROMPT

translator = OllamaTranslator(**translator_kw)

if not translator.check_connection():
    raise RuntimeError(
        "Не удалось подключиться к Ollama. Убедитесь, что Ollama запущен: ollama serve"
    )

available = translator.list_models()
if MODEL not in available:
    print(f"Предупреждение: модель '{MODEL}' не в списке. Доступные: {', '.join(available) or 'нет'}")

processor = PandocWordProcessor()
processor.max_chars = MAX_CHARS

print("Разбиваем документ на чанки...")
chunks = processor.chunk(input_path)
print(f"Создано чанков: {len(chunks)}")
print("-" * 50)

In [ ]:
def progress_callback(current: int, total: int, status: str):
    print(f"  [{current}/{total}] {status}")

print("Перевод...")
translated_chunks = translator.translate_batch(
    chunks,
    LANGUAGE,
    progress_callback=progress_callback,
)
translator.close()

In [ ]:
output_path = input_path.with_stem(f"{input_path.stem}_translated_{LANGUAGE}")
print("Собираем документ...")
processor.concatenate(translated_chunks, output_path)
print("-" * 50)
print(f"Готово! Результат сохранён: {output_path}")